In [1]:
from snap.snap import snapmerchants, cull_data
import snow.utils as sf
import pandas as pd
import editdistance

In [2]:
snap_df, count_error, name_error, size_of_data = snapmerchants('sydney'
                                                               ,sanity_check=True)

Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Custom data:  (630701, 1394, 'Melbourne', 'Dragon Hot Pot', 'Kingsway, Glen Waverley') 

Connection succesfully closed.
Using zans_role...
Established connection.
Querying zans_wh.zans_db.glue
Custom data:  (1365.0, 'https://www.zomato.com/sydney/sandoitchi-darlinghurst\n') 

Connection succesfully closed.
Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Custom data:  ('Balkan Seafood', 'Darlinghurst', '-33.88119518547905', '151.21783929064785', 'Sydney', 355) 

Connection succesfully closed.
Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Custom data:  (1211, 'Sydney') 

Connection succesfully closed.
We had 0 errors with extracting coordinates.



Is
Chilli and Spicy
Hot And Spicy House a match?
Respond with enter, or n for false-positives. If you make an error, type 'restart'.

 n
Is
Sergio's Trattoria and Pizzeria
Alberto's Pizzeria a match?
Respond with enter, or n for false-positives. If you make an error, type 'restart'.

 n
Is
Sergio's Trattoria and Pizzeria
Ladda's The Thai Takeaway a match?
Respond with enter, or n for false-positives. If you make an error, type 'restart'.

 n
Is
Sergio's Trattoria and Pizzeria
Miss Lilly's Kitchen a match?
Respond with enter, or n for false-positives. If you make an error, type 'restart'.

 n
Is
Kawa Ramen
K'square a match?
Respond with enter, or n for false-positives. If you make an error, type 'restart'.

 n
Is
Chinatown Noodle Restaurant
Northern Chinese Noodle Restaurant a match?
Respond with enter, or n for false-positives. If you make an error, type 'restart'.

 n
Is
Restaurant Moon
Yama Gardens Japanese Restaurant a match?
Respond with enter, or n for false-positives. If you make

In [4]:
snap_df

In [1]:
import pandas as pd
import numpy as np
from snap.snap import populate_empty_branches, cull_data

In [2]:
snap_df = pd.read_csv('snapped.csv')
df,interactions = populate_empty_branches(snap_df)
snapped_df = pd.concat([snap_df,df])

Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Interactions data:  ('fen_job@yahoo.com', '400958', '2019-06-13', None, None, None, Decimal('7.000000'), 'Yes', 'Sydney', 861, 'CoCo Fresh Tea & Juice', 1242, '2019-06-13', Decimal('7.000000000000')) 

Connection succesfully closed.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
interactions = interactions.rename({'merchant':'Name'},axis=1)
interactions = cull_data(interactions)
int_df = interactions[['branchId','merchantId','Name']].drop_duplicates()
int_df = int_df.rename({'branchId':'Id','merchantId':'MId','merchant':'Name'},axis=1)

In [7]:
final_df = snapped_df.merge(int_df,how='outer')
final_df = final_df.drop(['Suburb','Zomato Name','levenshtein','n'],axis=1)
final_df.url = final_df.url.replace(np.nan,'None')

In [8]:
template_df = final_df[final_df.url=='None']
template_df[['MId','Name','url']
           ].drop_duplicates(subset=['MId']
                ).to_csv('snap/manual/template_sydney.csv',index=False)

In [10]:
import os
x_df = pd.read_csv('snap/manual/sydney.csv')

In [105]:
url_dict = {}
for _, r in x_df.iterrows():
    key = int(r['MId'])
    url_dict[key] = r['url']

In [127]:
df = pd.DataFrame(zip(list(url_dict.keys())
                 ,list(url_dict.values()))
                 ,columns=['MId','url'])

In [20]:
y = x_df.merge(final_df,on='MId')

In [12]:
y[y.Name_x!=y.Name_y]

,MId,Name_x,url_x,Id,Name_y,url_y


In [29]:
snapped = final_df[final_df.url!='None']
COMPLETE = pd.concat([snapped[['Id','url']],y[['Id','url_x']].rename({'url_x':'url'},axis=1)])

In [30]:
import snow.utils as sf

In [32]:
sf.to_snow(df=COMPLETE.rename({'Id':'branchId'},axis=1)
           ,table='sydney'
           ,schema='glue')

2019-06-14
Accepted dataframe object
Schema: glue created.
sydney successfully updated.
Engine successfully closed
